In [999]:
import numpy as np
import matplotlib.pyplot as plt

In [1000]:
from loaders import clean_text_for_pattern_letter
from temp_text import template_str
input_str = clean_text_for_pattern_letter(template_str, out_len=500)
print(len(input_str))
letters = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
               'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', '\'', '1']
print(f'{letters=}')

len(letters)=32
500
letters=['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '.', ',', '"', "'", '1']


In [1001]:
def predict_train(input_str, letters, max_index:int = 4,):
    next_index: dict[str, int] = {}
    index_by_prev: dict[str, dict] = {}

    StatsDict: dict[str, dict] = {}

    for letter in letters:
        next_index[letter] = 1

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_idx = inp_idx - 1
        prev_inp = input_str[prev_idx]
        
        inp_index = next_index[inp]

        prev_index = next_index[prev_inp]

        stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
        StatsDict.setdefault(stats_key, {})

        next_inp = input_str[inp_idx + 1]
        StatsDict[stats_key].setdefault(next_inp, 0)

        StatsDict[stats_key][next_inp] += 1

        by_prev_key = prev_inp + str(prev_index) + ':' + inp
        index_by_prev.setdefault(by_prev_key, {})
        index_by_prev[by_prev_key].setdefault(inp_index, 0)
        index_by_prev[by_prev_key][inp_index] += 1

        by_prev_key = prev_inp + str(0) + ':' + inp
        index_by_prev.setdefault(by_prev_key, {})
        index_by_prev[by_prev_key].setdefault(inp_index, 0)
        index_by_prev[by_prev_key][inp_index] += 1
        
        next_index[prev_inp] += 1
        if next_index[prev_inp] > max_index:
            next_index[prev_inp] = 1
    
    return StatsDict, index_by_prev

In [1002]:
def predict(input_str, TF:list, StatsDict:dict, index_by_prev:dict, max_index, err = 0,):
    prev_index_l = [x for x in range(1, max_index+1)]

    output = []

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]
        
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index:
                max_key_d = 1
            
            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)
            # B B {'B': 1} [3] {3: 1} [3] False
            output.append(max_key)

        prev_index_l = list(index_d.keys())
        
        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)
    
    output.insert(0, input_str[1])
    
    return TF, err, output

In [1003]:
# train
max_index_L1 = 4

index_by_prev_L1: dict[str, dict] = {}
StatsDict_L1: dict[str, dict] = {}

StatsDict, index_by_prev_L1 = predict_train(letters=letters, input_str=input_str, max_index=max_index_L1)

In [1004]:
for sdk, sdv in index_by_prev_L1.items():
    print(sdk, sdv)
    print('')

t1:h {1: 2}

t0:h {1: 4, 2: 2, 3: 3, 4: 2}

h1:e {1: 1, 4: 3}

h0:e {1: 1, 3: 3, 4: 5, 2: 4}

e1:  {1: 2, 4: 1}

e0:  {1: 6, 3: 4, 4: 4, 2: 3}

 1:s {1: 1}

 0:s {1: 1, 3: 2, 4: 1, 2: 2}

s1:k {1: 1}

s0:k {1: 1}

k1:y {1: 1}

k0:y {1: 1}

y1:  {2: 1}

y0:  {2: 1, 1: 2, 3: 1}

 2:a {1: 1}

 0:a {1: 3, 3: 5, 2: 2, 4: 2}

a1:b {1: 1}

a0:b {1: 1}

b1:o {1: 1}

b0:o {1: 1, 2: 2}

o1:v {1: 1}

o0:v {1: 1}

v1:e {2: 1}

v0:e {2: 1, 3: 2, 1: 1}

e2:  {3: 3, 2: 1, 1: 2}

 3:t {2: 2, 3: 1, 4: 1}

 0:t {2: 5, 4: 3, 1: 3, 3: 3}

t2:h {2: 2, 1: 2, 4: 1}

h2:e {3: 2, 2: 2}

e3:  {4: 2, 1: 1, 2: 2}

 4:p {1: 1}

 0:p {1: 2, 4: 1}

p1:o {2: 1}

p0:o {2: 1}

o2:r {1: 2}

o0:r {1: 3, 2: 2, 4: 1}

r1:t {3: 1}

r0:t {3: 1}

t3:  {1: 1}

t0:  {1: 2, 4: 1}

 1:w {1: 1}

 0:w {1: 1, 2: 2, 4: 2, 3: 2}

w1:a {2: 1}

w0:a {2: 3, 1: 2}

a2:s {2: 2, 1: 1}

a0:s {2: 2, 1: 3, 3: 3}

s2:  {2: 1, 4: 1}

s0:  {2: 2, 4: 7, 1: 2, 3: 3}

 2:t {4: 1, 2: 1, 3: 2}

t4:h {3: 2, 4: 1}

h3:e {4: 2, 2: 1}

e4:  {3: 1, 1: 1, 4

In [1005]:
for sdk, sdv in StatsDict.items():
    print(sdk, sdv)
    print('')

t1:h1 {'e': 2}

h1:e1 {' ': 1}

e1: 1 {'s': 1, 'd': 1}

 1:s1 {'k': 1}

s1:k1 {'y': 1}

k1:y1 {' ': 1}

y1: 2 {'a': 1}

 2:a1 {'b': 1}

a1:b1 {'o': 1}

b1:o1 {'v': 1}

o1:v1 {'e': 1}

v1:e2 {' ': 1}

e2: 3 {'t': 1, 's': 1, 'c': 1}

 3:t2 {'h': 2}

t2:h2 {'e': 2}

h2:e3 {' ': 2}

e3: 4 {'p': 1, 'f': 1}

 4:p1 {'o': 1}

p1:o2 {'r': 1}

o2:r1 {'t': 1, 'd': 1}

r1:t3 {' ': 1}

t3: 1 {'w': 1}

 1:w1 {'a': 1}

w1:a2 {'s': 1}

a2:s2 {' ': 1, 'e': 1}

s2: 2 {'t': 1}

 2:t4 {'h': 1}

t4:h3 {'e': 1, 'i': 1}

h3:e4 {' ': 1, 'a': 1}

e4: 3 {'c': 1}

 3:c1 {'o': 1}

c1:o3 {'l': 1}

o3:l1 {'o': 1}

l1:o4 {'r': 1}

o4:r2 {' ': 2}

r2: 4 {'o': 1}

 4:o1 {'f': 1}

o1:f1 {' ': 1}

f1: 1 {'t': 1}

 1:t1 {'e': 1}

t1:e1 {'l': 1, 's': 1}

e1:l2 {'e': 1}

l2:e2 {'v': 1}

e2:v2 {'i': 1, 'e': 1}

v2:i1 {'s': 1}

i1:s3 {'i': 1}

s3:i2 {'o': 1}

i2:o2 {'n': 1}

o2:n1 {',': 1, 'o': 1}

n1:,1 {' ': 1}

,1: 2 {'t': 1}

 2:t2 {'u': 1}

t2:u1 {'n': 1}

u1:n2 {'e': 1}

n2:e3 {'d': 1}

e3:d1 {' ': 1}

d1: 3 {'t': 1}



In [1006]:
# predict
TF = []

TF, err, out_L1 = predict(input_str=input_str, TF=TF, StatsDict=StatsDict, index_by_prev=index_by_prev_L1, max_index=max_index_L1)

print(err, err / (len(input_str) / 100) )

e e {'e': 4} [1, 2, 4, 3] {1: 4, 2: 2, 4: 2, 3: 3} [1, 2, 3, 4] False
    {' ': 4, 'a': 1, 't': 1, 'r': 1} [1, 4, 3, 2] {1: 1, 4: 5, 3: 3, 2: 4} [1, 2, 4, 3] False
s s {'s': 2, 'd': 2, 'm': 1, 'c': 1} [1, 4, 3, 2] {1: 6, 4: 4, 3: 4, 2: 3} [1, 4, 3, 2] False
p k {'k': 1, 'a': 1, 'o': 1, 'p': 2, 'h': 1} [1, 4, 3, 2] {1: 1, 4: 1, 3: 2, 2: 2} [1, 4, 3, 2] False
y y {'y': 1} [1] {1: 1} [1, 4, 3, 2] False
    {' ': 1} [1] {1: 1} [1] False
a a {'a': 1} [2] {2: 1} [1] False
b b {'b': 1} [1] {1: 1} [2] False
o o {'o': 1} [1] {1: 1} [1] False
v v {'v': 1} [1] {1: 1} [1] False
e e {'e': 1} [1] {1: 1} [1] False
    {' ': 1} [2] {2: 1} [1] False
c t {'t': 1, 's': 1, 'c': 2, 'i': 1, 'f': 1} [3, 2, 1] {3: 3, 2: 1, 1: 2} [2] False
h h {'h': 2, 'o': 1, 'e': 1} [2, 3, 4, 1] {2: 4, 3: 3, 4: 3, 1: 1} [3, 2, 1] False
e e {'e': 4, 'r': 1} [2, 1, 4, 3] {2: 2, 1: 4, 4: 2, 3: 3} [2, 3, 4, 1] False
    {' ': 4, 'r': 1, 'a': 1, 't': 1} [3, 2, 1, 4] {3: 3, 2: 4, 1: 1, 4: 5} [2, 1, 4, 3] False
s p {'p': 1, 'f': 1,

In [1007]:
TF = np.array(TF, dtype=float)
# print(TF)
TF = np.pad(TF, (1, 1), constant_values=[1,1])
# print(TF)
TF = np.pad(TF, (1, 0))[:-1] - TF
# TF = TF - np.pad(TF, (1, 0))[:-1] 
# TF = np.pad(TF, (1, 0))[:-1] - TF
# TF = TF - np.pad(TF, (0, 1))[-1:]
# print(TF)
TF[np.where(TF < 0)] = 0
# TF = np.abs(TF)
TF[np.where(TF <= np.mean(TF[np.where(TF > 0)]))] = 0
# # TF[np.where(TF <= 0.5)] = 0
# print(TF.size, TF.max(), TF.min())

# fig, axs = plt.subplots(1, 1, figsize=(50, 5), tight_layout=True)
# # axs.plot(np.arange(TF.size), TF)
# markerline, stemline, baseline, = axs.stem(np.arange(TF.size), TF)
# plt.setp(markerline, markersize = 1)
# axs.xaxis.set_ticks(np.arange(len(input_str)) ,input_str)
# # axs.xaxis.grid(True)

# plt.show()

In [1008]:
upstream_inp: dict[int, list] = {}

temp_seq = []

for inp_idx, inp in enumerate(input_str):
    tf = TF[inp_idx]
    
    if tf > 0:
        upstream_inp.setdefault(inp_idx, temp_seq.copy())
        temp_seq.clear()

    temp_seq.append(inp)

for item, val in upstream_inp.items():
    print(item, val)
    
print(len(upstream_inp))

5 ['t', 'h', 'e', ' ', 's']
18 ['k', 'y', ' ', 'a', 'b', 'o', 'v', 'e', ' ', 't', 'h', 'e', ' ']
32 ['p', 'o', 'r', 't', ' ', 'w', 'a', 's', ' ', 't', 'h', 'e', ' ', 'c']
37 ['o', 'l', 'o', 'r', ' ']
53 ['o', 'f', ' ', 't', 'e', 'l', 'e', 'v', 'i', 's', 'i', 'o', 'n', ',', ' ', 't']
59 ['u', 'n', 'e', 'd', ' ', 't']
66 ['o', ' ', 'a', ' ', 'd', 'e', 'a']
72 ['d', ' ', 'c', 'h', 'a', 'n']
116 ['n', 'e', 'l', '.', ' ', '"', 'i', 't', "'", 's', ' ', 'n', 'o', 't', ' ', 'l', 'i', 'k', 'e', ' ', 'i', "'", 'm', ' ', 'u', 's', 'i', 'n', 'g', ',', '"', ' ', 'c', 'a', 's', 'e', ' ', 'h', 'e', 'a', 'r', 'd', ' ', 's']
124 ['o', 'm', 'e', 'o', 'n', 'e', ' ', 's']
135 ['a', 'y', ',', ' ', 'a', 's', ' ', 'h', 'e', ' ', 's']
155 ['h', 'o', 'u', 'l', 'd', 'e', 'r', 'e', 'd', ' ', 'h', 'i', 's', ' ', 'w', 'a', 'y', ' ', 't', 'h']
166 ['r', 'o', 'u', 'g', 'h', ' ', 't', 'h', 'e', ' ', 'c']
175 ['r', 'o', 'w', 'd', ' ', 'a', 'r', 'o', 'u']
183 ['n', 'd', ' ', 't', 'h', 'e', ' ', 'd']
187 ['o', 'o', 'r',

In [1009]:
letters_dict: dict[str, int] = {}
for let_idx, letter in enumerate(letters):
    letters_dict.setdefault(letter, let_idx)

upstream_patterns = []

for upstream_letters in upstream_inp.values():
    upstream_pattern = np.zeros(len(letters))

    for letter in upstream_letters:
        pattern_idx = letters_dict.get(letter, 0)

        upstream_pattern[pattern_idx] = 1
    
    upstream_patterns.append(upstream_pattern)

upstream_patterns = np.array(upstream_patterns)

for up_ptr in upstream_patterns:
    print(up_ptr.astype(int).tolist())

[0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0]
[0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0]
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 1, 0, 0, 1, 0, 0,

In [1010]:
layer_size = 16

I = np.zeros(upstream_patterns.shape[1])
O = np.zeros(layer_size)
W = np.ones((O.size, I.size))
Trace = np.zeros((O.size, I.size))
print('W', W.shape, W.max(), W.min(), np.median(W))

k = 1
push_delta = 0.4

prec = 1e-30

epoch_cnt = 100
eps0 = 0.02

eps = 0.05
rate_drop = 0.0

AVG_O = np.zeros(O.shape)
AVG_O_CNT = np.zeros(O.shape)

samples_idx = np.arange(upstream_patterns.shape[0])

for epoch_idx in range(epoch_cnt):
    np.random.shuffle(samples_idx)

    for i_idx in samples_idx:
        I = upstream_patterns[i_idx]

        # infer
        O = np.dot(W, I)

        # train
        winner_ids = np.argsort(O)[::-1][:1+k]
        winner = winner_ids[0]
        loosers = winner_ids[1:1+k]

        # update
        W[winner] = (1 - eps) * W[winner] + eps * (I - 0.5) + Trace[winner]*eps
        W[winner] = np.clip(W[winner], -1, 1)

        # W[loosers] = (1 - eps) * W[loosers] + (eps * (I - 0.5) * -push_delta) + Trace[loosers]*eps
        # W[loosers] = np.clip(W[loosers], -1, 1)

        # update Trace
        rate_Trace = np.zeros(I.shape)
        rate_Trace[(I > 0.5)] = 0.3
        rate_Trace[(I < 0.5)] = -0.001

        dTrace = np.zeros(W.shape)
        dTrace[winner] = rate_Trace
        Trace += dTrace
        Trace = np.clip(Trace, 0, 1)

        eps -= eps * rate_drop

print('W', W.shape, W.max(), W.min(), np.median(W))

W (16, 32) 1.0 1.0 1.0
W (16, 32) 1.0 -0.4999999999999995 -0.275828514347339


In [1011]:
SL_symbols = [x.capitalize() for x in letters][:layer_size]
print(len(SL_symbols))

SLSymbolDict = {}

SL_symbols_d = {}
for idx, val in enumerate(SL_symbols):
    SL_symbols_d.setdefault(val + ':' + str(idx), 0)

upstream_inp_keys = list(upstream_inp.keys())

upstream_cluster_by_symbol: dict[str, int] = {}
for idx, val in enumerate(SL_symbols):
    upstream_cluster_by_symbol.setdefault(val, idx)

for upstr_idx, up_ptr in enumerate(upstream_patterns):
    O = np.dot(W, up_ptr)

    winner = np.argmax(O)
    symbol = SL_symbols[winner]
    print(SL_symbols[winner])

    # print(W[upstream_cluster_by_symbol[symbol]].tolist())

    sls = SL_symbols[winner]
    sls_key = upstream_inp_keys[upstr_idx]
    SLSymbolDict.setdefault(sls_key, sls)
    
    SL_symbols_d[SL_symbols[winner] + ':' + str(winner)] += 1

print(len(SL_symbols_d))
for key, val in SL_symbols_d.items():
    print(key, val)

print(SLSymbolDict)

16
D
K
D
B
F
D
C
O
D
D
H
D
M
G
D
B
M
D
D
D
P
D
P
J
K
D
A
A
N
I
E
L
D
H
16
A:0 2
B:1 2
C:2 1
D:3 13
E:4 1
F:5 1
G:6 1
H:7 2
I:8 1
J:9 1
K:10 2
L:11 1
M:12 2
N:13 1
O:14 1
P:15 2
{5: 'D', 18: 'K', 32: 'D', 37: 'B', 53: 'F', 59: 'D', 66: 'C', 72: 'O', 116: 'D', 124: 'D', 135: 'H', 155: 'D', 166: 'M', 175: 'G', 183: 'D', 187: 'B', 198: 'M', 237: 'D', 266: 'D', 311: 'D', 355: 'P', 375: 'D', 384: 'P', 392: 'J', 404: 'K', 415: 'D', 429: 'A', 436: 'A', 455: 'N', 459: 'I', 465: 'E', 468: 'L', 484: 'D', 495: 'H'}


In [1012]:
sl_input = list(SLSymbolDict.values())
print(sl_input)

['D', 'K', 'D', 'B', 'F', 'D', 'C', 'O', 'D', 'D', 'H', 'D', 'M', 'G', 'D', 'B', 'M', 'D', 'D', 'D', 'P', 'D', 'P', 'J', 'K', 'D', 'A', 'A', 'N', 'I', 'E', 'L', 'D', 'H']


In [1013]:
# SL train
max_index_L2 = 4

StatsDict_L2, index_by_prev_L2 = predict_train(input_str=sl_input, letters=SL_symbols, max_index=max_index_L2)

In [1014]:
for sdk, sdv in StatsDict_L2.items():
    print(sdk, sdv)
    print('')

D1:K1 {'D': 1}

K1:D2 {'B': 1}

D2:B1 {'F': 1}

B1:F1 {'D': 1}

F1:D3 {'C': 1}

D3:C1 {'O': 1}

C1:O1 {'D': 1}

O1:D4 {'D': 1}

D4:D4 {'H': 1, 'D': 1}

D1:H1 {'D': 1}

H1:D2 {'M': 1}

D2:M1 {'G': 1}

M1:G1 {'D': 1}

G1:D3 {'B': 1}

D3:B2 {'M': 1}

B2:M2 {'D': 1}

M2:D4 {'D': 1}

D1:D1 {'P': 1}

D2:P1 {'D': 1}

P1:D3 {'P': 1}

D3:P2 {'J': 1}

P2:J1 {'K': 1}

J1:K2 {'D': 1}

K2:D4 {'A': 1}

D4:A1 {'A': 1}

A1:A1 {'N': 1}

A2:N1 {'I': 1}

N1:I1 {'E': 1}

I1:E1 {'L': 1}

E1:L1 {'D': 1}

L1:D1 {'H': 1}



In [1015]:
for sdk, sdv in index_by_prev_L2.items():
    print(sdk, sdv)
    print('')

D1:K {1: 1}

D0:K {1: 1}

K1:D {2: 1}

K0:D {2: 1, 4: 1}

D2:B {1: 1}

D0:B {1: 1, 2: 1}

B1:F {1: 1}

B0:F {1: 1}

F1:D {3: 1}

F0:D {3: 1}

D3:C {1: 1}

D0:C {1: 1}

C1:O {1: 1}

C0:O {1: 1}

O1:D {4: 1}

O0:D {4: 1}

D4:D {4: 2}

D0:D {4: 2, 1: 1}

D1:H {1: 1}

D0:H {1: 1}

H1:D {2: 1}

H0:D {2: 1}

D2:M {1: 1}

D0:M {1: 1}

M1:G {1: 1}

M0:G {1: 1}

G1:D {3: 1}

G0:D {3: 1}

D3:B {2: 1}

B2:M {2: 1}

B0:M {2: 1}

M2:D {4: 1}

M0:D {4: 1}

D1:D {1: 1}

D2:P {1: 1}

D0:P {1: 1, 2: 1}

P1:D {3: 1}

P0:D {3: 1}

D3:P {2: 1}

P2:J {1: 1}

P0:J {1: 1}

J1:K {2: 1}

J0:K {2: 1}

K2:D {4: 1}

D4:A {1: 1}

D0:A {1: 1}

A1:A {1: 1}

A0:A {1: 1}

A2:N {1: 1}

A0:N {1: 1}

N1:I {1: 1}

N0:I {1: 1}

I1:E {1: 1}

I0:E {1: 1}

E1:L {1: 1}

E0:L {1: 1}

L1:D {1: 1}

L0:D {1: 1}



In [1016]:
# predict
TF_L2 = []

TF_L2, err_L2, out_L2 = predict(input_str=sl_input, 
                                TF=TF_L2, 
                                StatsDict=StatsDict_L2, 
                                index_by_prev=index_by_prev_L2, 
                                max_index=max_index_L2)

for o in out_L2:
    print(o)

print(err_L2, err_L2 / (len(sl_input) / 100), len(sl_input), len(out_L2) )

D D {'D': 1} [1] {1: 1} [1, 2, 3, 4] False
B B {'B': 1} [2] {2: 1} [1] False
F F {'F': 1} [1] {1: 1} [2] False
D D {'D': 1} [1] {1: 1} [1] False
C C {'C': 1} [3] {3: 1} [1] False
O O {'O': 1} [1] {1: 1} [3] False
D D {'D': 1} [1] {1: 1} [1] False
D D {'D': 1} [4] {4: 1} [1] False
H H {'H': 1, 'D': 1} [4] {1: 2} [4] True
D D {'D': 1} [1] {1: 1} [1] False
M M {'M': 1} [2] {2: 1} [1] False
G G {'G': 1} [1] {1: 1} [2] False
D D {'D': 1} [1] {1: 1} [1] False
B B {'B': 1} [3] {3: 1} [1] False
M M {'M': 1} [2] {2: 1} [3] False
D D {'D': 1} [2] {2: 1} [2] False
D D {'D': 1} [4] {4: 1} [2] False
H D {'H': 1, 'D': 1} [4] {1: 2} [4] True
P P {'P': 1} [1] {2: 1} [1] True
D D {'D': 1} [1] {1: 1} [2] False
P P {'P': 1} [3] {3: 1} [1] False
J J {'J': 1} [2] {2: 1} [3] False
K K {'K': 1} [1] {1: 1} [2] False
D D {'D': 1} [2] {2: 1} [1] False
A A {'A': 1} [4] {4: 1} [2] False
A A {'A': 1} [1] {1: 1} [4] False
N N {'N': 1} [1] {2: 1} [1] True
I I {'I': 1} [1] {1: 1} [2] False
E E {'E': 1} [1] {1: 1} [1]

In [1017]:
def ds_predict_train_without_index(input_str: str | list,
                                   #    sl_input: list,
                                   SLSymbolDict: dict,
                                   #    StatsDict_L1: dict,
                                   #    index_by_prev_L1: dict,
                                   #    max_index_L1: int,
                                   ):
    ds_symbol = '0'
    DsSymbolsStats = {}
    DsSymbolsStatsProb = {}

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        peak_symbol = SLSymbolDict.get(inp_idx, 0)  # inp_idx - 1
        if peak_symbol != 0:
            ds_symbol = peak_symbol

        DsSymbolsStats.setdefault(ds_symbol, {})
        DsSymbolsStats[ds_symbol].setdefault(inp, 0)

        DsSymbolsStats[ds_symbol][inp] += 1

    for ds_inp, inps in DsSymbolsStats.items():
        norm = sum(inps.values())

        for symbol, symbol_stats in inps.items():
            DsSymbolsStatsProb[ds_inp + ':' + symbol] = symbol_stats / norm

    return DsSymbolsStats, DsSymbolsStatsProb

In [1018]:
DsSymbolsStats, DsSymbolsStatsProb = ds_predict_train_without_index(input_str, SLSymbolDict)
print(DsSymbolsStatsProb)

{'0:h': 0.25, '0:e': 0.25, '0: ': 0.25, '0:s': 0.25, 'D:k': 0.009478672985781991, 'D:y': 0.018957345971563982, 'D: ': 0.1943127962085308, 'D:a': 0.11374407582938388, 'D:b': 0.014218009478672985, 'D:o': 0.07582938388625593, 'D:v': 0.014218009478672985, 'D:e': 0.0947867298578199, 'D:t': 0.037914691943127965, 'D:h': 0.03317535545023697, 'D:l': 0.023696682464454975, 'D:r': 0.05687203791469194, 'D:d': 0.018957345971563982, 'D:m': 0.004739336492890996, 'D:n': 0.023696682464454975, 'D:s': 0.07582938388625593, 'D:,': 0.004739336492890996, 'D:u': 0.014218009478672985, 'D:g': 0.009478672985781991, 'D:c': 0.023696682464454975, 'D:i': 0.03317535545023697, 'D:f': 0.023696682464454975, 'D:.': 0.014218009478672985, 'D:"': 0.004739336492890996, 'D:w': 0.023696682464454975, 'D:p': 0.023696682464454975, 'D:j': 0.009478672985781991, 'D:x': 0.004739336492890996, 'K:p': 0.04, 'K:o': 0.12, 'K:r': 0.08, 'K:t': 0.08, 'K: ': 0.2, 'K:w': 0.12, 'K:a': 0.04, 'K:s': 0.08, 'K:h': 0.04, 'K:e': 0.04, 'K:c': 0.04, 'K:

In [1019]:
# from itertools import cycle

def ds_predict(input_str, 
               StatsDict: dict, 
               index_by_prev: dict,
               upstream_cluster_by_symbol: dict,
               letters_dict: dict,
               out_L2: list,
               SLSymbolDict: dict,
               W,
               DsSymbolsStats: dict,
               max_index):
    prev_index_l = [x for x in range(1, max_index+1)]

    err = 0
    TF = []
    output = []

    ds_out_idx = 0
    ds_inp = '0'
    # ds_pattern = np.ones(W.shape[1])

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]
        
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = prev_inp + str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        # downstream
        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if ds_out_idx == len(out_L2):
                ds_inp = '0'
                # ds_pattern = np.ones(W.shape[1])
            else:
                ds_inp = out_L2[ds_out_idx]
                # ds_pattern = W[upstream_cluster_by_symbol.get(ds_inp)]
                # ds_pattern = np.copy(W[upstream_cluster_by_symbol.get(ds_inp)])
                # ds_pattern[np.where(ds_pattern < 0)] = 0

                ds_out_idx += 1
        print(stats_d)

        # prob stats
        norm = sum(stats_d.values())
        for st_key, st in stats_d.items():
            stats_d[st_key] = st / norm

        for st_key, st in stats_d.items():
            # letter_idx = letters_dict.get(st_key)
            
            # ds_stats = ds_pattern[letter_idx]
            ds_key = ds_inp + ':' + st_key
            ds_stats = DsSymbolsStats.get(ds_key, 0)
            print('st_ds:', st_key, st, ds_key, ds_stats)
            if ds_stats <= 0:
                stats_d[st_key] = 0

                continue
            
            stats_d[st_key] *= ds_stats
            print('upd:', ds_stats)
        
        print(stats_d)
        print(input_str[inp_idx + 1], max(stats_d, key=stats_d.get))
        print()

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index:
                max_key_d = 1
            
            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            # print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)
            output.append(max_key)

        prev_index_l = list(index_d.keys())
        
        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)
    
    output.insert(0, input_str[1])
    
    return TF, err, output

In [1020]:
DsSymbolsStats, DsSymbolsStatsProb = ds_predict_train_without_index(input_str, SLSymbolDict)

dsp_TF, dsp_err, dsp_output = ds_predict(input_str=input_str,
                                         StatsDict=StatsDict,
                                         index_by_prev=index_by_prev_L1,
                                         upstream_cluster_by_symbol=upstream_cluster_by_symbol,
                                         letters_dict=letters_dict,
                                         out_L2=out_L2,
                                         SLSymbolDict=SLSymbolDict,
                                         W=W,
                                         DsSymbolsStats=DsSymbolsStatsProb,
                                         max_index=max_index_L1)

print(dsp_err, dsp_err / (len(input_str) / 100))


{'e': 4}
st_ds: e 1.0 0:e 0.25
upd: 0.25
{'e': 0.25}
e e

{' ': 4, 'a': 1, 't': 1, 'r': 1}
st_ds:   0.5714285714285714 0:  0.25
upd: 0.25
st_ds: a 0.14285714285714285 0:a 0
st_ds: t 0.14285714285714285 0:t 0
st_ds: r 0.14285714285714285 0:r 0
{' ': 0.14285714285714285, 'a': 0, 't': 0, 'r': 0}
   

{'s': 2, 'd': 2, 'm': 1, 'c': 1}
st_ds: s 0.3333333333333333 0:s 0.25
upd: 0.25
st_ds: d 0.3333333333333333 0:d 0
st_ds: m 0.16666666666666666 0:m 0
st_ds: c 0.16666666666666666 0:c 0
{'s': 0.08333333333333333, 'd': 0, 'm': 0, 'c': 0}
s s

{'k': 1, 'a': 1, 'o': 1, 'p': 2, 'h': 1}
st_ds: k 0.16666666666666666 0:k 0
st_ds: a 0.16666666666666666 0:a 0
st_ds: o 0.16666666666666666 0:o 0
st_ds: p 0.3333333333333333 0:p 0
st_ds: h 0.16666666666666666 0:h 0.25
upd: 0.25
{'k': 0, 'a': 0, 'o': 0, 'p': 0, 'h': 0.041666666666666664}
k h

{'y': 1}
st_ds: y 1.0 K:y 0
{'y': 0}
y y

{' ': 1}
st_ds:   1.0 K:  0.2
upd: 0.2
{' ': 0.2}
   

{'a': 1}
st_ds: a 1.0 K:a 0.04
upd: 0.04
{'a': 0.04}
a a

{'b': 1}
st_d

In [1021]:
ds_predict_train_without_index(input_str, SLSymbolDict)

({'0': {'h': 1, 'e': 1, ' ': 1, 's': 1},
  'D': {'k': 2,
   'y': 4,
   ' ': 41,
   'a': 24,
   'b': 3,
   'o': 16,
   'v': 3,
   'e': 20,
   't': 8,
   'h': 7,
   'l': 5,
   'r': 12,
   'd': 4,
   'm': 1,
   'n': 5,
   's': 16,
   ',': 1,
   'u': 3,
   'g': 2,
   'c': 5,
   'i': 7,
   'f': 5,
   '.': 3,
   '"': 1,
   'w': 5,
   'p': 5,
   'j': 2,
   'x': 1},
  'K': {'p': 1,
   'o': 3,
   'r': 2,
   't': 2,
   ' ': 5,
   'w': 3,
   'a': 1,
   's': 2,
   'h': 1,
   'e': 1,
   'c': 1,
   'd': 1,
   'i': 1,
   'n': 1},
  'B': {'o': 3,
   'f': 2,
   ' ': 4,
   't': 4,
   'e': 3,
   'l': 1,
   'v': 1,
   'i': 2,
   's': 1,
   'n': 1,
   ',': 1,
   'h': 2,
   'c': 1,
   'a': 1},
  'F': {'u': 1, 'n': 1, 'e': 1, 'd': 1, ' ': 1, 't': 1},
  'C': {'d': 1, ' ': 1, 'c': 1, 'h': 1, 'a': 1, 'n': 1},
  'O': {'n': 3,
   'e': 4,
   'l': 2,
   '.': 1,
   ' ': 8,
   '"': 2,
   'i': 4,
   't': 2,
   "'": 2,
   's': 4,
   'o': 1,
   'k': 1,
   'm': 1,
   'u': 1,
   'g': 1,
   ',': 1,
   'c': 1,
   'a': 2,
  

In [1022]:
def downstream_predict_train(input_str: str | list,
                             sl_input: list,
                             SLSymbolDict: dict,
                            #  ds_out: list,
                             StatsDict_L1: dict,
                            #  StatsDict_L2,
                            #  TF_L1,
                             index_by_prev_L1: dict,
                            #  index_by_prev_L2,
                             max_index_L1: int,
                            #  max_index_L2,
                             ):
    next_index: dict[str, int] = {}

    for letter in letters:
        next_index[letter] = 1

    ds_inp = '0'
    sl_inp_idx = 1

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_idx = inp_idx - 1
        prev_inp = input_str[prev_idx]

        inp_index = next_index[inp]

        prev_index = next_index[prev_inp]

        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if sl_inp_idx == len(sl_input):
                ds_inp = '0'
            else:
                ds_inp = sl_input[sl_inp_idx]
                sl_inp_idx += 1
        # print(f'{ds_inp=}')
        stats_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp + str(inp_index)
        StatsDict_L1.setdefault(stats_key, {})

        next_inp = input_str[inp_idx + 1]
        StatsDict_L1[stats_key].setdefault(next_inp, 0)

        StatsDict_L1[stats_key][next_inp] += 1

        by_prev_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp
        index_by_prev_L1.setdefault(by_prev_key, {})
        index_by_prev_L1[by_prev_key].setdefault(inp_index, 0)
        index_by_prev_L1[by_prev_key][inp_index] += 1

        by_prev_key = ds_inp + ':' + prev_inp + str(0) + ':' + inp
        index_by_prev_L1.setdefault(by_prev_key, {})
        index_by_prev_L1[by_prev_key].setdefault(inp_index, 0)
        index_by_prev_L1[by_prev_key][inp_index] += 1

        next_index[prev_inp] += 1
        if next_index[prev_inp] > max_index_L1:
            next_index[prev_inp] = 1
        
    return StatsDict_L1, index_by_prev_L1


In [1023]:
StatsDict_L1_ds = {}
index_by_prev_L1_ds = {}

StatsDict_L1_ds, index_by_prev_L1_ds = downstream_predict_train(input_str=input_str,
                                                                sl_input=sl_input,
                                                                SLSymbolDict=SLSymbolDict,
                                                                StatsDict_L1=StatsDict_L1_ds,
                                                                index_by_prev_L1=index_by_prev_L1_ds,
                                                                max_index_L1=max_index_L1, )

for key, val in StatsDict_L1_ds.items():
    print(key, val)

0:t1:h1 {'e': 1}
0:h1:e1 {' ': 1}
0:e1: 1 {'s': 1}
0: 1:s1 {'k': 1}
K:s1:k1 {'y': 1}
K:k1:y1 {' ': 1}
K:y1: 2 {'a': 1}
K: 2:a1 {'b': 1}
K:a1:b1 {'o': 1}
K:b1:o1 {'v': 1}
K:o1:v1 {'e': 1}
K:v1:e2 {' ': 1}
K:e2: 3 {'t': 1}
K: 3:t2 {'h': 1}
K:t2:h2 {'e': 1}
K:h2:e3 {' ': 1}
K:e3: 4 {'p': 1}
D: 4:p1 {'o': 1}
D:p1:o2 {'r': 1}
D:o2:r1 {'t': 1, 'd': 1}
D:r1:t3 {' ': 1}
D:t3: 1 {'w': 1}
D: 1:w1 {'a': 1}
D:w1:a2 {'s': 1}
D:a2:s2 {' ': 1, 'e': 1}
D:s2: 2 {'t': 1}
D: 2:t4 {'h': 1}
D:t4:h3 {'e': 1, 'i': 1}
D:h3:e4 {' ': 1}
D:e4: 3 {'c': 1}
D: 3:c1 {'o': 1}
B:c1:o3 {'l': 1}
B:o3:l1 {'o': 1}
B:l1:o4 {'r': 1}
B:o4:r2 {' ': 1}
B:r2: 4 {'o': 1}
F: 4:o1 {'f': 1}
F:o1:f1 {' ': 1}
F:f1: 1 {'t': 1}
F: 1:t1 {'e': 1}
F:t1:e1 {'l': 1}
F:e1:l2 {'e': 1}
F:l2:e2 {'v': 1}
F:e2:v2 {'i': 1}
F:v2:i1 {'s': 1}
F:i1:s3 {'i': 1}
F:s3:i2 {'o': 1}
F:i2:o2 {'n': 1}
F:o2:n1 {',': 1}
F:n1:,1 {' ': 1}
F:,1: 2 {'t': 1}
F: 2:t2 {'u': 1}
D:t2:u1 {'n': 1}
D:u1:n2 {'e': 1}
D:n2:e3 {'d': 1}
D:e3:d1 {' ': 1}
D:d1: 3 {'t': 1}
D: 3:t3

In [1024]:
from itertools import cycle

print(out_L2)
def downstream_predict(input_str: str | list,
                       SLSymbolDict: dict,
                       StatsDict_L1_ds: dict,
                       index_by_prev_L1_ds: dict,
                       out_L2: list,
                       max_index_L1: int,
                       TF: list,
                       err=0,):
    prev_index_l = [x for x in range(1, max_index_L1+1)]

    ds_inp = '0'
    ds_out_idx = 0
    ds_predict_out = cycle(out_L2)

    for inp_idx, inp in enumerate(input_str):
        if inp_idx <= 0 or inp_idx == len(input_str) - 1:
            continue

        prev_inp = input_str[inp_idx - 1]

        # print(inp, SLSymbolDict.get(inp_idx, 0))
        is_peak = SLSymbolDict.get(inp_idx, 0)
        if is_peak != 0:
            if ds_out_idx == len(out_L2):
                ds_inp = '0'
            else:
                ds_inp = out_L2[ds_out_idx]
                ds_out_idx += 1
        print(ds_inp, is_peak)
        index_l = []
        stats_d = {}
        index_d = {}
        for prev_index in prev_index_l:
            by_prev_key = ds_inp + ':' + prev_inp + str(prev_index) + ':' + inp
            # print(by_prev_key)
            inp_index_arr = index_by_prev_L1_ds.get(by_prev_key, {-1: -1})
            # print(inp_index_arr)

            for inp_index, val in inp_index_arr.items():
                if inp_index != -1:
                    index_d.setdefault(inp_index, 0)
                    index_d[inp_index] += val

                if inp_index != -1 and index_l.__contains__(inp_index) == False:
                    index_l.append(inp_index)

                    stats_key = ds_inp + ':' + prev_inp + \
                        str(prev_index) + ':' + inp + str(inp_index)
                    stats = StatsDict_L1_ds.get(stats_key, {})

                    next_inp = input_str[inp_idx + 1]
                    # print(next_inp, stats_key, stats)
                    for st_key, st in stats.items():
                        stats_d.setdefault(st_key, 0)
                        stats_d[st_key] += 1

        max_key_d = max(index_d, key=index_d.get)
        if inp == prev_inp:
            pred_max_key = max_key_d
            max_key_d += 1
            if max_key_d > max_index_L1:
                max_key_d = 1

            index_d[max_key_d] = index_d[pred_max_key]
            index_d.pop(pred_max_key)

        if len(stats_d) > 0:
            max_key = max(stats_d, key=stats_d.get)
            # print(max_key, next_inp, stats_d, index_l, index_d, prev_index_l, inp == prev_inp)

        prev_index_l = list(index_d.keys())

        if max_key != next_inp:
            err += 1

        tf = len(index_l)
        TF.append(tf)

    return TF, err


['K', 'D', 'B', 'F', 'D', 'C', 'O', 'D', 'D', 'H', 'D', 'M', 'G', 'D', 'B', 'M', 'D', 'D', 'H', 'P', 'D', 'P', 'J', 'K', 'D', 'A', 'A', 'N', 'I', 'E', 'L', 'D', 'H']


In [1025]:
TF_L1_ds = []

TF_L1_ds, err_L1_ds = downstream_predict(input_str=input_str,
                                         SLSymbolDict=SLSymbolDict,
                                         StatsDict_L1_ds=StatsDict_L1_ds,
                                         index_by_prev_L1_ds=index_by_prev_L1_ds,
                                         out_L2=out_L2,
                                         max_index_L1=max_index_L1,
                                         TF=TF_L1_ds, )

print(err_L1_ds, err_L1_ds / (len(input_str) / 100))

0 0
0 0
0 0
0 0
K D
K 0
K 0
K 0
K 0
K 0
K 0
K 0
K 0
K 0
K 0
K 0
K 0
D K
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
B D
B 0
B 0
B 0
B 0
F B
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
F 0
D F
D 0
D 0
D 0
D 0
D 0
C D
C 0
C 0
C 0
C 0
C 0
C 0
O C
O 0
O 0
O 0
O 0
O 0
D O
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D D
D 0
D 0
D 0
D 0
D 0
D 0
D 0
H D
H 0
H 0
H 0
H 0
H 0
H 0
H 0
H 0
H 0
H 0
D H
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
M D
M 0
M 0
M 0
M 0
M 0
M 0
M 0
M 0
M 0
M 0
G M
G 0
G 0
G 0
G 0
G 0
G 0
G 0
G 0
D G
D 0
D 0
D 0
D 0
D 0
D 0
D 0
B D
B 0
B 0
B 0
M B
M 0
M 0
M 0
M 0
M 0
M 0
M 0
M 0
M 0
M 0
D M
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D D
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0
D 0


ValueError: max() arg is an empty sequence